In [1]:
import pandas as pd
import numpy as np
import csv
from bs4 import BeautifulSoup
import requests

In [2]:
# Mountain Project provides a csv when filtering through boulders
# This csv provides 1000 climbs rated first by popularity and then by difficulty.
# csv written to dataframe for exploration.
df = pd.read_csv('JoshuaTreeBoulders1.5PLus.csv')
df.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'JoshuaTreeBoulders1.5PLus.csv'

In [ ]:
# Checking that full location is provided
temp = df[df['Route'] == 'White Crack']['Location'].item()
temp

In [ ]:
# Mountain project includes descriptions on individual boulder pages. 
# We want to extract the boulders' descriptions and add it to the dataframe for inspection.
descriptions = []
for index, row in df.iterrows():
    temp_link = row['URL']
    response = requests.get(temp_link)
    temp_soup = BeautifulSoup(response.content, "html.parser")
    html_description = temp_soup.find_all("div", {"class": "fr-view"})
    descriptions.append(html_description)

df['Description'] = descriptions

df.head()


#temp_link = df[df['Route'] == 'White Rastafarian']['URL'].item()
#response = requests.get(temp_link)
#temp_soup = BeautifulSoup(response.content, "html.parser")
#html_desc = temp_soup.find_all("div", {"class": "fr-view"})
#print(html_desc)
#descriptions.append(html_desc)

In [ ]:
#Cool, Seems like the descriptions are okay, some might have more content than necessary, but it works for now.
#Now we want to create dummy variables for key words like caves, roofs, crimps, etc so we need to parse through descriptions

#This function finds a key word (feature) in the description and creates a column with dummy variables
def has_feature(column_name, feature, data):
    temp_list = []
    for index, row in data.iterrows():
        if feature in str(row['Description']).lower():
            temp_list.append(1)
        else:
            temp_list.append(0)
    data[column_name] = temp_list

In [ ]:
# List of hold types we want to consider
feature_list = ['crimp', 'sloper', 'traverse', 'arete', 'jug', 'undercling', 'cave', 'roof', 'pinch', 'mantle']
for feature in feature_list:
    has_feature('Has'+ feature, feature, df)
    
for i in range(len(df)):
    if df.loc[i, 'Rating'] == 'V-easy' or 'V-easy R':
        df.loc[i, 'Rating'] = 0
    else:
        df.loc[i, 'Rating'] = re.findall(r'\d+', df.loc[i, 'Rating'])
df.head()

In [244]:
print(re.findall(r'\d+',  df[df['Route'] == 'White Rastafarian']['Rating'].item()))

['2']


In [13]:
df.to_csv(r'C:\Users\Noah Kim\Downloads\testgpt.csv')

df = pd.read_csv('testgpt.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'testgpt.csv'

In [232]:
#Lets do some clustering
import re
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Drop any irrelevant columns or handle missing values if needed
mldata = df.copy()
mldata = mldata.dropna()  # Example: dropping rows with missing values
mldata = mldata.drop(columns = ['Route', 'Location', 'URL', 'Your Stars', 'Route Type', 'Area Latitude', 'Area Longitude', 'Description'])



                

# Standardize the data (important for distance-based algorithms)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(mldata)

# Set the number of clusters (choose based on your data or experiment with it)
kmeans = KMeans(n_clusters=3, random_state=42)
mldata['Cluster'] = kmeans.fit_predict(data_scaled)

# View the clustered data
print(mldata.head())

# Reduce to 2D for visualization
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_scaled)

plt.scatter(data_pca[:, 0], data_pca[:, 1], c=mldata['Cluster'], cmap='viridis')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('KMeans Clustering')
plt.colorbar()
plt.show()

ValueError: could not convert string to float: 'V2 R'